<a href="https://colab.research.google.com/github/joshuamach632-hub/TB-model/blob/main/GRADCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/TB_Model/tb_cnn_model.h5')

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    47,776,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,795,908 (182.33 MB)

 Trainable params: 47,795,906 (182.33 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
for layer in model.layers:
    print(layer.name)

conv2d
max_pooling2d
conv2d_1
max_pooling2d_1
flatten
dense
dense_1


In [ ]:
#Dependencies

import os
import json
from zipfile import ZipFile
from PIL import Image

import numpy as np
import matplotlib.pyplot as PLT
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [ ]:
!pip install kaggle

In [ ]:
kaggle_credentials = json.load(open("kaggle.json"))

In [ ]:
#setup Kaggle API key as environment variables
os.environ['KAGGLE USERNAME'] = kaggle_credentials["username"]
os.environ['KAGGLE KEY'] = kaggle_credentials["key"]

In [27]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tawsifurrahman/tuberculosis-tb-chest-xray-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/tuberculosis-tb-chest-xray-dataset


In [28]:
!cp -r /kaggle/input/tuberculosis-tb-chest-xray-dataset /content/

In [29]:
!ls /content/tuberculosis-tb-chest-xray-dataset

TB_Chest_Radiography_Database


In [30]:
last_conv_layer_name = 'conv2d_1'

In [43]:
#GRAD CAM FUNCTION
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Call the model once to define its output shape
    _ = model(img_array)

    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [32]:
# Set up ImageDataGenerator for training data
train_datagen = ImageDataGenerator(rescale=1./255) # Add other augmentations here if needed

# Create the train_generator
train_generator = train_datagen.flow_from_directory(
    '/content/tuberculosis-tb-chest-xray-dataset/TB_Chest_Radiography_Database/', # Path to your training data
    target_size=(224, 224), # Set this to the input size of your model
    batch_size=32,
    class_mode='categorical' # Or 'binary' depending on your model's output layer
)

Found 4200 images belonging to 2 classes.


In [33]:
#function to load and preprocess image using pillow

def load_and_preprocess_image(image_path, target_size=(224, 224)):

  #load image
  from PIL import Image
  img= Image.open(image_path)

  #Convert to RGB using PIL
  img = Image.open(image_path).convert('RGB')

  #Resize image
  img= img.resize(target_size)

  #Convert image into numpy array

  img_array= np.array(img)

  #add batch dimension
  img_array= np.expand_dims(img_array, axis=0)

  #scale image values to 0,1

  img_array= img_array.astype('float32')/255.
  return img_array

In [34]:
#function to predict class of an image

def predict_image_class(model, image_path, class_indices):

 preprocessed_img = load_and_preprocess_image(image_path)
 predictions = model.predict(preprocessed_img)
 predicted_class_index= np.argmax(predictions, axis=1) [0]
 predicted_class_name = class_indices[predicted_class_index]
 return predicted_class_name

In [35]:
#creating a napping from class indices to class names
class_indices={v:k for k, v in train_generator.class_indices.items()}

In [36]:
#saving the class names as a json file
json.dump(class_indices, open('class_indices.json', 'w'))

In [37]:
#Example
#image_path="/content/76052f7902246ff862f52f5d3cd9cd_big_gallery.jpg"
#image_path="/content/pulmonary-tuberculosis-29.jpg"
#image_path="/content/pulmonary-tuberculosis-67.png"
image_path="/content/tuberculosis-tb-chest-xray-dataset/TB_Chest_Radiography_Database/Tuberculosis/Tuberculosis-10.png"
predicted_class_name = predict_image_class(model, image_path,class_indices)

#output the result

print( "X-ray detected:", predicted_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
X-ray detected: Tuberculosis


In [38]:
#Load and preprocess image

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

# Load image
img_path = "/content/tuberculosis-tb-chest-xray-dataset/TB_Chest_Radiography_Database/Tuberculosis/Tuberculosis-1.png"
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalize

#activate model
_ = model.predict(img_array)  # or model(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [42]:
heatmap = make_gradcam_heatmap(img_array, model, 'conv2d_1')

# Convert heatmap to RGB
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Load original image again (in case it's needed for overlay)
original_img = cv2.imread(img_path)
original_img = cv2.resize(original_img, (512, 512))

# Superimpose
superimposed_img = cv2.addWeighted(original_img, 0.6, heatmap, 0.4, 0)

# Save or display
plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Grad-CAM')
plt.show()

AttributeError: The layer sequential has never been called and thus has no defined output.

In [45]:
heatmap = make_gradcam_heatmap(img_array, model, 'conv2d_1')

# Convert heatmap to RGB
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Load original image again (in case it's needed for overlay)
original_img = cv2.imread(img_path)
original_img = cv2.resize(original_img, (224, 224)) # Resize original image to match heatmap size

# Superimpose
superimposed_img = cv2.addWeighted(original_img, 0.6, heatmap, 0.4, 0)

# Save or display
plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Grad-CAM')
plt.show()

AttributeError: The layer sequential has never been called and thus has no defined output.